In [4]:
import pickle

#opening the pickle file
file_name = "pickle_files/dtw60.pkl"
with open(file_name, 'rb') as file:
    cleaned_sp500_df = pickle.load(file)


In [5]:
cleaned_sp500_df.head()


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
#n_series contains the number of time series 
n_series=cleaned_sp500_df.shape[1]
n_series

In [ ]:
import numpy as np
# Initializing distance matrix
euclidean_distance = np.zeros((n_series, n_series))
dtw0_distance = np.zeros((n_series, n_series))
dtw60_distance = np.zeros((n_series, n_series))

In [ ]:
from helpermodules.dtwclustering import DTWClustering

In [ ]:
# Initializing distance matrix
euclidean_distance = np.zeros((n_series, n_series))
dtw0_distance = np.zeros((n_series, n_series))
dtw60_distance = np.zeros((n_series, n_series))

#two for cycle in order to obtain a distance matrix
for i in range(n_series):
    for j in range(i, n_series):
        distance= DTWClustering(cleaned_sp500_df.iloc[:,i], cleaned_sp500_df.iloc[:,j])
        euclidean_distance[i,j]= distance.euclidean_distance()
        dtw0_distance[i,j]= distance.dtw_no_window()
        dtw60_distance[i,j]= distance.dtw_with_window()
        # copy to value because the matrix is symmetric
        euclidean_distance[j, i] = euclidean_distance[i, j]
        dtw0_distance[j, i] = dtw0_distance[i, j]
        dtw60_distance[j, i] = dtw60_distance[i, j]